In [1]:
from util import *
from search import *
from old_search import search as old_search, search_overlap as old_search_overlap, search_overlap2 as old_search_overlap_alt
from prototype import *
from random import shuffle

In [5]:
seqs = load_fasta_records_as_str("uniprot_sprot.fasta")
shuffle(seqs)
seqs = seqs[:10000]
peps = collapse_second_order_list(map(digest_trypsin,seqs))
peps = list(filter(lambda pep: 'X' not in pep, peps))
specs = list(map(generate_spectrum_and_list_mz,peps))
specs = [np.unique(spec) for spec in specs]
tolerance = 2 * AVERAGE_MASS_DIFFERENCE
alphabet = AMINO_MASS_MONO

In [6]:
from time import time
def duration(fn, specs, *args):
    init_t = time()
    for spec in specs:
        fn(spec,*args)
    return time() - init_t

In [7]:
duration(old_search_overlap,specs,tolerance,alphabet)

47.93575596809387

In [8]:
duration(old_search_overlap_alt,specs,tolerance,alphabet)

46.427249908447266

In [ ]:
duration(search,specs,"overlap",alphabet,tolerance)

In [ ]:
duration(search,specs,"overlap_alt",alphabet,tolerance)